In [ ]:
"""damped_pendulum.ipynb"""
# Cell 1


from __future__ import annotations

import typing

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import AutoMinorLocator
from scipy.integrate import solve_ivp  # type: ignore

if typing.TYPE_CHECKING:
    from typing import Any

    from matplotlib.axes import Axes
    from numpy.typing import NDArray

%matplotlib widget

# fmt: off
def model(time: float,state_vector: tuple[float, float],
    phase_constant: float, damping_constant: float) -> tuple[float, float]:
    omega: float
    theta: float
    omega, theta = state_vector  # unpack dependent variables
    d_omega: float = -damping_constant * omega - phase_constant * np.sin(theta)
    d_theta: float = omega
    return d_omega, d_theta
# fmt: on


def plot(ax: Axes) -> None:
    # Precalculate phase constant
    pendulum_length = 1.0  # meters
    phase_constant: float = 9.81 / pendulum_length

    # Set damping_constants
    underdamped_constant: float = 1.0
    overdamped_constant: float = pow(phase_constant, 2) / 2.0
    critically_damped_constant: float = pow(phase_constant, 2) / 4.0

    # Set initial conditions
    omega_initial: float = 0
    theta_initial: float = np.radians(75)  # 75 degrees

    # Set model duration (seconds)
    time_initial: float = 0
    time_final: float = 15

    # Calculate for an underdamped pendulum
    sol: Any = solve_ivp(
        model,
        (time_initial, time_final),
        [omega_initial, theta_initial],
        max_step=0.01,
        args=[phase_constant, underdamped_constant],
    )
    time_steps: NDArray[np.float_] = np.array(sol.t, dtype=np.float_)
    theta_underdamped: NDArray[np.float_] = np.array(sol.y[1], dtype=np.float_)

    # Calculate for an overdamped pendulum
    sol = solve_ivp(
        model,
        (time_initial, time_final),
        [omega_initial, theta_initial],
        max_step=0.01,
        args=[phase_constant, overdamped_constant],
    )
    theta_overdamped: NDArray[np.float_] = np.array(sol.y[1], dtype=np.float_)

    # Calculate for a critically damped pendulum
    sol = solve_ivp(
        model,
        (time_initial, time_final),
        [omega_initial, theta_initial],
        max_step=0.01,
        args=[phase_constant, critically_damped_constant],
    )
    theta_critically_damped: NDArray[np.float_] = np.array(sol.y[1], dtype=np.float_)

    # fmt: off
    ax.plot(time_steps, theta_underdamped, label="underdamped",
             color="red", linestyle="solid", linewidth=2, zorder=3)
    
    ax.plot(time_steps, theta_overdamped, label="overdamped",
             color="blue", linestyle="solid", linewidth=2, zorder=3)
    
    ax.plot(time_steps, theta_critically_damped, label="critically damped",
             color="green", linestyle="solid", linewidth=2, zorder=3)
    # fmt: on

    ax.set_title("Damped Pendulum")
    ax.set_xlabel("Time (sec)")
    ax.set_ylabel(r"$\theta$ (radians)")
    ax.axhline(y=0.0, color="lightgray")
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.legend(loc="upper right")


def main() -> None:
    plt.close("all")
    plt.figure(" ")
    plot(plt.axes())
    plt.show()


main()